# TENSORS

Tensors 是一个特殊的数据结构，它和数组和矩阵非常相似。在 PyTorch，我们使用 tensors 去编码模型的输入和输出，以及模型的参数

Tensors 与 [Numpy](https://numpy.org/) 的 ndarrays 非常相似，不过 tensors 可以运行在 GPU 或者其他的硬件加速器上。事实上， tensor 和 Numpy 的数据都经常共享底层的内存，[抵消了复制数据的需要](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)。Tensors 也为自动微分提供了优化。如果你熟悉 ndarrays，你在使用 Tensor 提供的 API 时就会感觉像回到了家一样。

In [1]:
import torch
import numpy as np

## 初始化 Tensor

Tensors 有多种初始化方式：

### 直接从数据创建

Tensors 可以直接从数据中创建。数据类型会被自动推断出来。

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

### 从 Numpy 数组中创建

Tensors 可以直接从 Numpy 数组中创建（反之亦然）。

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

### 从另一个 tensor 中创建：

新的张量保存着作为参数的张量的属性（shape, datatype）除非显示地覆盖。

In [5]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.5328, 0.8486],
        [0.7813, 0.3194]]) 



### 通过随机或者常量初始化

`shape` 是一个张量维度的元组。在下面的函数中，它决定了输出张量的维度。

In [6]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.7442, 0.9863, 0.0858],
        [0.4938, 0.2646, 0.1570]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## Tensor 的属性

张量的属性包括 shape, datatype 以及存储他们的设备

In [7]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 张量上的操作

超过一百个张量操作，包括算数，线性代数，矩阵计算（转制、索引、切片），采样

这些操作都运行在 GPU 上（通常比 CPU 上快）

默认情况下，张量创建在 CPU 上。我们需要显示地通过 `.to` 方法将张量移动到 GPU 上（需要检查一下设备是否有可用 GPU ）。需要记住跨设备复制大规模张量在时间和内存上的消耗都是十分昂贵的！

In [8]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

**标准的 numpy 形式的索引和切片**

In [9]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**连接张量** 你可以使用 `torch.cat` 沿给定的维数串联张量。

In [12]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**算术运算**

In [13]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**单元素张量** 如果你有一个单元素张量，比如通过将张量所有的值聚合到一个单独值上，你可以通过 `item()` 将其转换为 Python 中的数值

In [14]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


**原地操作** 直接将操作结果存储在操作数上的操作叫做原地操作。他们以 `_` 后缀来表示。比如： `x.copy_(y)`， `x.t_()` 将会改变 `x`

In [15]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


## 与 Numpy 的区别

CPU 上的张量和 NumPy 数组可以共享它们的底层内存位置，改变一个就会改变另一个。

In [16]:
## 将 Tensor 转换为 Numpy 数组

t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


对于 tensor 的改变会影响到 Numpy 数组：

In [17]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


## Numpy 数组转换为 Tensor

In [18]:
n = np.ones(5)
t = torch.from_numpy(n)

改变 Numpy 数组会影响到 tensor

In [19]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
